# MOFA+

In this notebook we "train" our MOFA model, in order to find the Latent Factors.

In [2]:
from mofapy2.run.entry_point import entry_point
import pandas as pd
#import cupy as cp
import numpy as np

In [2]:
#print(cp.cuda.runtime.getDeviceCount())
#print(cp.__version__)

1
13.6.0


In [3]:
ent = entry_point()


        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        


## Data Loading

In [4]:
GO_BP = pd.read_csv('data/gene_go_matrix_propT_rel-is_a-part_of_ont-BP.csv')
GO_CC = pd.read_csv('data/gene_go_matrix_propT_rel-is_a-part_of_ont-CC.csv')
GO_MF = pd.read_csv('data/gene_go_matrix_propT_rel-is_a-part_of_ont-MF.csv')
HPO = pd.read_csv('data/gene_hpo_matrix_binary_withAncestors_namespace_Phenotypic abnormality.csv')

In [5]:
GO_BP = GO_BP.values.astype(float)
GO_CC = GO_CC.values.astype(float)
GO_MF = GO_MF.values.astype(float)
HPO = HPO.values.astype(float)

In [6]:
print(GO_BP.shape)
print(GO_CC.shape)
print(GO_MF.shape)
print(HPO.shape)

(5183, 9874)
(5183, 1479)
(5183, 3259)
(5183, 10186)


In [7]:
D = [GO_BP.shape[1],GO_CC.shape[1], GO_MF.shape[1], HPO.shape[1]] # Number of features per view
M = len(D)      # Number of views
K = 15           # Number of factors
N = [5183] * 4   # Number of samples per group
G = len(N)      # Number of groups

In [8]:
data = [[GO_BP], [GO_CC], [GO_MF], [HPO]]

## Model Training

In [9]:
ent.set_data_options(scale_groups=False, scale_views=False)
ent.set_data_matrix(
    data=data,
    likelihoods=['bernoulli', 'bernoulli', 'bernoulli', 'bernoulli']
)
ent.set_model_options(factors=15)

View names not provided, using default naming convention:
- view1, view2, ..., viewM

Features names not provided, using default naming convention:
- feature1_view1, featureD_viewM

Groups names not provided, using default naming convention:
- group1, group2, ..., groupG

Samples names not provided, using default naming convention:
- sample1_group1, sample2_group1, sample1_group2, ..., sampleN_groupG

Successfully loaded view='view0' group='group0' with N=5183 samples and D=9874 features...
Successfully loaded view='view1' group='group0' with N=5183 samples and D=1479 features...
Successfully loaded view='view2' group='group0' with N=5183 samples and D=3259 features...
Successfully loaded view='view3' group='group0' with N=5183 samples and D=10186 features...


Model options:
- Automatic Relevance Determination prior on the factors: False
- Automatic Relevance Determination prior on the weights: True
- Spike-and-slab prior on the factors: False
- Spike-and-slab prior on the weights: Tr

In [10]:
ent.set_train_options(iter=100, convergence_mode='slow', gpu_mode=True)


GPU mode is activated, but GPU not found... switching to CPU mode
For GPU mode, you need:
1 - Make sure that you are running MOFA+ on a machine with an NVIDIA GPU
2 - Install CUPY following instructions on https://docs-cupy.chainer.org/en/stable/install.html



In [11]:
ent.build()
ent.run()

KeyboardInterrupt: 

## Saving the Model

In [16]:
ent.save(outfile='model/mofa.hdf5', save_data=True)

Saving model in data/mofa.hdf5...
